In [1]:
import os

import pandas as pd
import numpy as np

from sklearn import preprocessing
# from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Get PATH folder from previous directorys
def get_path(prev_folders:int=0):
    for i in range(prev_folders-1): os.chdir('../')  # Change to previous folder
    PATH = os.path.dirname(os.getcwd()) + '/'
    PATH = PATH.replace('\\', '/')
    return PATH

# Environment settings

In [3]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes
PATH = get_path(prev_folders=1)

# Read data

In [4]:
df_leads = pd.read_csv(PATH + 'data/leads.csv')
df_offers = pd.read_csv(PATH + 'data/offers.csv')

In [5]:
df_leads.sample(n=5).head()

,Id,First Name,Use Case,Source,Status,Discarded/Nurturing Reason,Acquisition Campaign,Created Date,Converted,City
29839,NaN,NaN,Sports Events,Inbound,New,NaN,NaN,2019-02-23,0,NaN
1165,NaN,Reese1 Thompson1,Corporate Events,Inbound,New,Not Fit,NaN,2018-10-24,0,San Francisco
16033,oz4un75g,NaN,Corporate Events,Inbound,Nurturing,Not feeling,NaN,2019-01-08,0,New York
30460,78q72j4v,Reese2 Thomas10,Corporate Events,Inbound,Nurturing,Unreachable,Follow-up: digital guide 2,2018-12-29,0,Chicago
913,NaN,NaN,Educational Seminars,Inbound,Nurturing,Competitor,NaN,2018-10-23,0,San Francisco


In [6]:
df_offers.sample(n=5).head()

,Id,Use Case,Status,Created Date,Close Date,Price,Discount code,Pain,Loss Reason
1126,n0f9x45e,Sports Events,Closed Won,2018-07-17,2018-07-17,1008.0,RENEWYEAR30,operations,NaN
883,t3i8uiwm,Sports Events,Closed Won,2018-12-02,2018-12-02,576.0,HALFSEASON20G,financial control,NaN
1509,7eja0psv,Educational Seminars,Closed Lost,2018-06-02,2018-09-02,1008.0,NaN,quality of delivery,no decision
5644,NaN,Corporate Events,Closed Lost,2019-03-19,2019-03-18,960.0,ANNUAL60DEAL,operations,no response
421,lwrwwa9u,Corporate Events,Closed Lost,2019-11-25,2019-11-25,360.0,MONTHSAVE25DEAL,financial control,no response


# EDA

In [7]:
df_offers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130 entries, 0 to 6129
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             4962 non-null   object 
 1   Use Case       6093 non-null   object 
 2   Status         6130 non-null   object 
 3   Created Date   6130 non-null   object 
 4   Close Date     6130 non-null   object 
 5   Price          5765 non-null   float64
 6   Discount code  4431 non-null   object 
 7   Pain           5261 non-null   object 
 8   Loss Reason    2364 non-null   object 
dtypes: float64(1), object(8)
memory usage: 431.1+ KB


In [8]:
df_leads.isnull().sum()

Id                            17667
First Name                     8399
Use Case                      16721
Source                         1346
Status                            0
Discarded/Nurturing Reason    16470
Acquisition Campaign          36592
Created Date                      0
Converted                         0
City                          26710
dtype: int64

In [9]:
df_offers.isnull().sum()

Id               1168
Use Case           37
Status              0
Created Date        0
Close Date          0
Price             365
Discount code    1699
Pain              869
Loss Reason      3766
dtype: int64

# Preprocesing

In [10]:
print(df_leads.shape)
print(df_offers.shape)

df_leads = df_leads.dropna(subset=['Id']).reset_index(drop=True)  # Delete Id's with NaN
df_offers = df_offers.drop_duplicates()

print(df_leads.shape)
print(df_offers.shape)

(61639, 10)
(6130, 9)
(43972, 10)
(6076, 9)


In [11]:
df_leads['Converted'] = df_leads['Converted'].replace({
    0:'No',
    1:'Yes'
    })

In [12]:
df_leads = df_leads.rename({
    'Status':'Status_leads'
    }, axis=1
    )

In [13]:
columns_drop = [
    'Use Case',
    'Created Date'
]

for column in columns_drop:
    df_leads.drop([column], axis=1, inplace=True)

## Merge

In [14]:
df = df_offers.merge(df_leads, on='Id', how='left')

# Delete variables to free memory
del df_offers
del df_leads

In [15]:
df['Has discount'] = df['Discount code'].isnull().apply(lambda x: not x)
df[['Has discount', 'Discount code']].sample(n=10).head(10)

,Has discount,Discount code
5745,True,2SEMESTERS20OFF
5527,True,2SEMESTERS20OFF
4029,True,SAVEMONTH50G
4516,True,2SEMESTERS20OFF
5583,False,NaN
3630,True,YEARLY30DISCOUNT
4480,False,NaN
3827,True,2SEMESTERS20OFF
5463,True,HALFYEAR20G
2384,True,MONTH75SAVINGS


In [16]:
df['Created Date'] = pd.to_datetime(df['Created Date'], format='%Y-%m-%d')
df['Close Date'] = pd.to_datetime(df['Close Date'], format='%Y-%m-%d')

df['Days in process'] = (df['Close Date'] - df['Created Date']) / np.timedelta64(1, 'D')

## Imputation

In [17]:
# Label Encoding
label_encoder = preprocessing.LabelEncoder()

columns_categorical = [
    'Use Case', 'Pain', 'Loss Reason', 'Source',
    'Status_leads', 'Discarded/Nurturing Reason',
    'Converted', 'City', 'Has discount'
]

columns_encoder = columns_categorical.copy()
for i in range(len(columns_encoder)):
    columns_encoder[i] = columns_encoder[i] + '_ENC'

# Label encoding ignoring null values
df[columns_encoder] = df[columns_categorical].apply(lambda series: pd.Series(
    label_encoder.fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

In [18]:
# Imputation with mean for numerical variables
df['Price'].fillna(df['Price'].mean(), inplace=True)

In [19]:
imputation_categorical = IterativeImputer(
    estimator=RandomForestClassifier(),
    initial_strategy='most_frequent',
    max_iter=10, random_state=0
    )

df[columns_encoder] = imputation_categorical.fit_transform(df[columns_encoder])

c:\Users\almontao\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


# Save

In [20]:
# Sort Columns in a specific order
df = df[[
    'Id', 'Use Case', 'Use Case_ENC', 'Status', 'Created Date', 'Close Date', 
    'Price', 'Discount code', 'Pain', 'Pain_ENC', 'Loss Reason', 'Loss Reason_ENC', 
    'First Name', 'Source','Source_ENC', 'Status_leads','Status_leads_ENC', 
    'Discarded/Nurturing Reason', 'Discarded/Nurturing Reason_ENC', 
    'Acquisition Campaign', 'Converted', 'Converted_ENC', 'City', 'City_ENC', 
    'Has discount', 'Has discount_ENC', 'Days in process'
]]

In [21]:
df.sample(n=5).head()

,Id,Use Case,Use Case_ENC,Status,Created Date,Close Date,Price,Discount code,Pain,Pain_ENC,Loss Reason,Loss Reason_ENC,First Name,Source,Source_ENC,Status_leads,Status_leads_ENC,Discarded/Nurturing Reason,Discarded/Nurturing Reason_ENC,Acquisition Campaign,Converted,Converted_ENC,City,City_ENC,Has discount,Has discount_ENC,Days in process
2426,NaN,Corporate Events,1.0,Closed Won,2019-06-05,2019-10-05,360.0,MONTHSAVE25DEAL,financial control,0.0,NaN,4.0,NaN,NaN,0.0,NaN,4.0,NaN,4.0,NaN,NaN,1.0,NaN,2.0,True,1.0,122.0
3225,4ityuvsd,Corporate Events,1.0,Closed Won,2018-11-21,2018-11-21,768.0,2SEMESTERS20OFF,operations,1.0,NaN,4.0,NaN,Inbound,0.0,Qualified,4.0,NaN,5.0,NaN,Yes,1.0,Chicago,2.0,True,1.0,0.0
4564,v4ck9swr,Sports Events,3.0,Closed Won,2019-01-08,2019-01-08,240.0,ECONOMY50GIFT,operations,1.0,NaN,4.0,NaN,Inbound,0.0,Qualified,4.0,NaN,5.0,NaN,Yes,1.0,NaN,2.0,True,1.0,0.0
299,t7n4mkpb,Sports Events,3.0,Closed Lost,2019-12-07,2019-12-09,150.0,MONTH75SPECIAL,operations,1.0,no budget,1.0,Morgan10 Jackson1,Inbound,0.0,Qualified,4.0,NaN,6.0,NaN,Yes,1.0,San Diego,15.0,True,1.0,2.0
1256,ik8jdbp1,Corporate Events,1.0,Closed Won,2018-07-08,2018-07-08,360.0,MONTHSAVE25DEAL,operations,1.0,NaN,1.0,NaN,Inbound,0.0,Qualified,4.0,NaN,6.0,NaN,Yes,1.0,San Francisco,16.0,True,1.0,0.0


In [22]:
df.to_csv(PATH + 'data/output_preprocesing.csv', index=False, encoding='utf-8')
# df.to_excel(PATH + 'data/output_preprocesing.xlsx', index=False)